In [ ]:
import face_recognition
import cv2
import os
import numpy as np
import serial
import time
from playsound import playsound

# Initialize serial communication with ESP32
ser = serial.Serial('COM7', 9600)  # Replace 'COM7' with your own ESP32 port
time.sleep(2)  # Allow time for connection to establish

# Load authorized face encodings
authorized_folder = "C:/Users/YEOH WAY HUP/Downloads/CPC357/authorised_faces" # Change path according to where the folder is stored
authorized_encodings = []  # List to store face encodings for authorized users
authorized_names = []      # List to store names of authorized users

# Load all authorized faces from the folder
for file in os.listdir(authorized_folder):
    image_path = os.path.join(authorized_folder, file)
    image = face_recognition.load_image_file(image_path)
    encodings = face_recognition.face_encodings(image)
    if encodings:  # Ensure at least one encoding is found
        authorized_encodings.append(encodings[0])
        authorized_names.append(os.path.splitext(file)[0])  # Extract file name as user name

print(f"Loaded authorized users: {authorized_names}")

# Function to play alarm sound for intruder detection
def sound_alarm():
    playsound('C:/Users/YEOH WAY HUP/Downloads/CPC357/alarm sound/Intruder Alarm Sound Effect.wav') # Change path according to where the file is stored

# Variable to monitor motion status
motion_detected = False

# Main loop
while True:
    # Check for messages from Arduino (whether motion is detected or not)
    if ser.in_waiting > 0:
        arduino_message = ser.readline().decode().strip()

        # Activate camera on motion detected
        if arduino_message == "Motion detected! Waiting for verification...":
            motion_detected = True  
            print("Motion detected! Activating camera...")

        # Deactivate camera on no motion detected
        elif arduino_message == "NO_MOTION":
            motion_detected = False
            print("No motion detected. Camera deactivated.")

    # Activate camera if motion is detected
    if motion_detected:
        camera = cv2.VideoCapture(0)

        while motion_detected:
            ret, frame = camera.read()
            if not ret:
                break

            # Process camera frame
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces and encode them
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # Default status is LOCK unless verified
            person_status = "LOCK"  
            intruder_detected = False  # Flag for intruder detection

            # Compare detected faces with authorized faces
            for face_encoding, face_location in zip(face_encodings, face_locations):

                # Compute distances to authorized encodings
                face_distances = face_recognition.face_distance(authorized_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)

                # Verify if face matches any authorized user
                if face_distances[best_match_index] < 0.5: 
                    name = authorized_names[best_match_index]
                    person_status = "UNLOCK"  # Change status to UNLOCK for authorized users
                else:
                    name = "Intruder"
                    intruder_detected = True  # Mark intruder as detected

                # Draw box around detected face
                top, right, bottom, left = face_location
                color = (0, 255, 0) if name != "Intruder" else (0, 0, 255)
                cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Send status to Arduino (UNLOCK or LOCK)
            ser.write((person_status + "\n").encode()) 
            print(f"Command sent: {person_status}")

            # Trigger alarm for intruder
            if intruder_detected:
                print("Intruder detected! Sounding alarm...")
                sound_alarm() 

            # Display video feed
            cv2.imshow("Home Security Camera", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Check for updates from Arduino to stop camera if no motion is detected
            if ser.in_waiting > 0:
                arduino_message = ser.readline().decode().strip()
                if arduino_message == "No motion detected. Locking door.":
                    print("No motion detected. Stopping camera...")
                    motion_detected = False
                    break

        # Release camera and close window
        camera.release()
        cv2.destroyAllWindows()

# Close serial connection
ser.close()